In [1]:
"""This code will crate a master table for each tissue"""
#Set up
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral":"orange",
    "Brain-Caudate":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed":"blue",
    "Skin-SunExposed":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m"
        }

SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral":"Adipose V.",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed": "Skin Unexposed",
    "Skin-SunExposed":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}

import math
import numpy as np
import os
import pandas as pd
import random
import sys
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

tissu = sorted(COLORS.keys())
CHR = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
Motifs = pd.read_csv('/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed', sep='\t', header=None)
Motifs.columns = ['chrom','str.start','str.end','motif.len','motif']
path='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'

In [3]:
A=[]
B=[]
All_eSTRs=[]
for Tissue in tissu: 
    command='mv '+path+Tissue+'/Master.table /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/old_Master.table'
    os.system(command)
    CAUSE=path+Tissue+'/HH/CAVIAR_analysis.table'
    SCORE=path+Tissue+'/HH/strs_scorechr'
    LR = path+Tissue+'/PQValues'
    ANOV = path+Tissue+'/HH/anova_wg.csv'
#LR
    lr = pd.read_csv(LR,sep='\t')
#Anova
    anova = pd.read_csv(ANOV, sep=',')
    anova = anova[['chrom','gene','str.start','anova_pval','estr_fdr','esnp_fdr','delta_bic','delta_aic']]
    #merge 1
    lr = pd.merge(lr, anova, on=['chrom','gene','str.start'], how='left')
#Causal 
    causal = pd.read_csv(CAUSE, sep='\t')
    causal['chrom']=causal['CHROM']
#Scores
    score_items=[]
    for i in CHR:
        C=pd.read_csv(SCORE+str(i), sep='\t', header=None)
        score_items.append(C)
    scores=score_items[0]
    for i in range(1, len(score_items)):
        scores=scores.append(score_items[i]) 
    scores.columns=['chrom', 'gene', 'str.id', 'caviar.score']
    CA = scores; CA.index = CA['gene']        
#Merge all in table
    Table = pd.merge(lr, CA, on=['chrom','gene','str.id'], how='left')
    Table.index = Table['gene']
    Master = pd.merge(Table,causal, on=['chrom','gene'], how='left')
    #print('With Caviar: ',Master.shape)
#Gene names
    genes=Gene_table.loc[Gene_table['gene'].isin(list(Master['gene']))]
    genes=genes[['gene','chrom','gene.name']] 
    Master=pd.merge(Master, genes, on=['chrom','gene'])
#motifs
    Master=pd.merge(Master, Motifs [['chrom','str.start','motif','str.end']], on=['chrom','str.start'])
#Additional columns
    Master['best.str'] = np.where(Master['qvalue']<=1, 1,0)
    Master['top.str.rank']=Master['str.rank']
    Master['top.snp.score']=Master['top_snp_score']
    Master['significant']= np.where(Master['qvalue']<=0.1, 1,0)
    Master['llsignificant'] = Master['llsignif'].astype(int)
    Master['top_str'] = Master['top_str'].astype(str)
    Master['top.variant'] = np.where(Master['top.str.score']>Master['top.snp.score'], Master['top_str'], Master['top_snp'])
    Master['best.str.start'] = Master['top_str'].astype(str).apply(lambda x: int(x.split('_')[1]) if 'STR' in x else '' )
#Summarize and savw
    Header = ['gene','chrom','str.start','str.id','best.str','best.str.start','top.variant','caviar.score','beta','beta.se','p.wald','qvalue','num.strs.in.top5','llqvalue','motif','top_str', 'top.str.score','top_snp', 'top.snp.score','top.str.rank','delta_bic','delta_aic','anova_pval','estr_fdr','esnp_fdr','llsignificant','significant', 'gene.name','str.end']
    Master = Master[Header].sort_values(['chrom','gene','str.start'])
    Master.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/Master.table', sep='\t',  index=None)
    #print('With gene name and motif : ',Master.shape, Tissue)
    
##   RUN The following
##   python ~/projects/GTEX_eSTRs/gtex-estrs/Scripts/STRs-vs-SNPs_effects/merge_causality_scores.py /storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/ WholeBlood,Cells-Transformedfibroblasts,Muscle-Skeletal,Lung,Artery-Tibial,Adipose-Subcutaneous,Esophagus-Mucosa causality > Merged_Best_causality.Table
##   To merge N select best by tissue
Master.transpose()

With gene name and motif :  (87892, 29) Adipose-Subcutaneous
With gene name and motif :  (87912, 29) Adipose-Visceral
With gene name and motif :  (86354, 29) Artery-Aorta
With gene name and motif :  (85367, 29) Artery-Tibial
With gene name and motif :  (87076, 29) Brain-Caudate
With gene name and motif :  (91146, 29) Brain-Cerebellum
With gene name and motif :  (83075, 29) Cells-Transformedfibroblasts
With gene name and motif :  (91510, 29) Esophagus-Mucosa
With gene name and motif :  (87610, 29) Esophagus-Muscularis
With gene name and motif :  (84416, 29) Heart-LeftVentricle
With gene name and motif :  (89449, 29) Lung
With gene name and motif :  (84241, 29) Muscle-Skeletal
With gene name and motif :  (88752, 29) Nerve-Tibial
With gene name and motif :  (89011, 29) Skin-NotSunExposed
With gene name and motif :  (89049, 29) Skin-SunExposed
With gene name and motif :  (89140, 29) Thyroid
With gene name and motif :  (83106, 29) WholeBlood


,0,5,9,12,14,19,24,28,1,6,...,81455,81459,81462,81365,81576,81579,81581,81381,81396,81413
gene,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,...,ENSG00000242616.3,ENSG00000242616.3,ENSG00000242616.3,ENSG00000244187.3,ENSG00000260230.2,ENSG00000260230.2,ENSG00000260230.2,ENSG00000268996.1,ENSG00000268996.1,ENSG00000268996.1
chrom,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,...,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9
str.start,169735051,169784101,169895824,169937973,169614029,169667612,169701424,169701512,169735051,169784101,...,114404884,114441796,114511252,139666301,111891767,111937749,111983014,139894040,140014401,140029812
str.id,STR_169735051,STR_169784101,STR_169895824,STR_169937973,STR_169614029,STR_169667612,STR_169701424,STR_169701512,STR_169735051,STR_169784101,...,STR_114404884,STR_114441796,STR_114511252,STR_139666301,STR_111891767,STR_111937749,STR_111983014,STR_139894040,STR_140014401,STR_140029812
best.str,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
best.str.start,169735051,169735051,169735051,169735051,169667612,169667612,169667612,169667612,169667612,169667612,...,114402178,114402178,114402178,139666301,111891767,111891767,111891767,140014401,140014401,140014401
top.variant,SNP_169860473,SNP_169860473,SNP_169860473,SNP_169860473,SNP_169756548,SNP_169756548,SNP_169756548,SNP_169756548,SNP_169756548,SNP_169756548,...,SNP_114420924,SNP_114420924,SNP_114420924,SNP_139677304,SNP_112008165,SNP_112008165,SNP_112008165,SNP_140038834,SNP_140038834,SNP_140038834
caviar.score,0.00171231,0.0014292,0.00140495,0.0014414,0.000514196,0.00251641,0.000614468,0.000490501,0.000477708,0.000386764,...,0.00126163,0.00274273,0.000780329,0.000149721,0.0038904,0.00162895,0.00228256,0.00144899,0.00220914,0.00162027
beta,0.044187,0.0179004,0.0131296,-0.019647,0.0517572,0.134485,-0.0636871,-0.0485744,0.0482733,-0.0205681,...,-0.0666603,-0.105726,-0.00742832,-0.0947223,0.0879198,-0.0326247,0.0592442,-0.00839826,0.0617559,-0.0325922
beta.se,0.0593858,0.0588141,0.059334,0.0581126,0.056179,0.0607569,0.0557882,0.056913,0.0593745,0.0588111,...,0.0598426,0.0589029,0.0600825,0.0606969,0.055512,0.0608257,0.0552876,0.0607435,0.0590184,0.0590998
